# Batch dispersion analysis

## Initialise session and load data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings                                  
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
import matplotlib.pyplot as plot
import matplotlib.colors

from src.data.data import *
data = data_wrapper()

Loading configuration file from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/src/config.yaml
Loaded configuration file
Loading devices data file from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/data/interim
Test initialisation done


In [3]:
# INPUT DATA
# Name of test to be analysed
dispersion_test = '2019-11_INT_8_FRAUNHOFER_8_STATION_ISCAPE'
# How to export the files ('png' or 'fig' or None)
type_file = None
# Percentage of points to be considered NG sensor
limit_errors = 3
# Multiplier for std_dev (sigma) - Normal distribution (99.73%)
limit_confidence_sigma = 3
# t-student confidence level (%)
t_confidence_level = 99
# Use average dispersion or instantaneous
use_instantatenous_dispersion = False
# Min/max date for the analysis
min_date = '2019-11-28'
max_date = '2019-12-02'
# In case there is a device with lower amount of channels, ignore the missing channels and keep going
ignore_missing_channels = True
# Smooth channels
smooth_channels = True
smooth_number = 20

In [7]:
data.load_test(dispersion_test)

Loading test 2019-11_INT_8_FRAUNHOFER_8_STATION_ISCAPE
No alphasense specified in files for 10408
No alphasense specified in files for 10409
No alphasense specified in files for 10412
No alphasense specified in files for 10413
No alphasense specified in files for 10414
No alphasense specified in files for 10420
No alphasense specified in files for 10421
No alphasense specified in files for 10422
Loaded cached info file
Loading device 9974
No metadata found - skipping
Requesting last reading from API for device 9974
Device 9974 has last reading at 2019-11-26T15:35:07Z
Dropping NaN
Loaded cached files from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/data/processed/2019/11/2019-11_INT_8_FRAUNHOFER_8_STATION_ISCAPE/cached/9974.csv
No need to load new data from API
Device 9974 has been loaded
Loading device 10408
No metadata found - skipping
Requesting last reading from API for device 10408
Device 10408 has last read


## Get list of common channels
Displays a warning in case there is a device that has fewer channels than the rest. You can choose whether or not to ignore it or update the list of common channels

In [ ]:
# Get list of common channels
save_path = join(data.dataDirectory, 'export/figs', dispersion_test)
# Test Path
if not exists(save_path):
    print ('Creating export directory:\n{}'.format(save_path))
    mkdir(save_path)

list_channels = list()
# Get list of devices
list_devices = list(data.tests[dispersion_test].devices.keys())

In [ ]:
# Init list of common channels. Get the one that has the most
list_channels = data.tests[dispersion_test].devices[list_devices[0]].readings.columns
# Extract list of common channels
len_channels = len(list_channels)

for device in list_devices:
    
    if ignore_missing_channels: 
        # We don't reduce the list in case the new list is smaller
        list_channels = list(set(list_channels) | set(data.tests[dispersion_test].devices[device].readings.columns))
    else:
        # We reduce it
        list_channels = list(set(list_channels) & set(data.tests[dispersion_test].devices[device].readings.columns))

    print ('Device {}'.format(device))
    print ('\tMin reading at {}'.format(data.tests[dispersion_test].devices[device].readings.index[0]))
    #min_date_records = min(min_date_records, records.readings[dispersion_test]['devices'][device]['data'].index[0])
    print ('\tMax reading at {}'.format(data.tests[dispersion_test].devices[device].readings.index[-1]))
    #max_date_records = min(max_date_records, records.readings[dispersion_test]['devices'][device]['data'].index[-1])
    print ('\tNumber of points {}'.format(len(data.tests[dispersion_test].devices[device].readings.index)))
    ## Eliminate devices with no points
    if (len(data.tests[dispersion_test].devices[device].readings.index) == 0):
        print ('Droping device {} for insufficient data points'.format(device))
        data.tests[dispersion_test].devices.pop(device)
    # Check the number of channels    
    elif len_channels != len(data.tests[dispersion_test].devices[device].readings.columns): 
        print("\tWARNING: Device {} has {}. Current common list length is {}".format(device, len(data.tests[dispersion_test].devices[device].readings.columns), len_channels))
        len_channels = len(list_channels)
        if ignore_missing_channels:
            print ("\tIgnoring missing channels")

print('Final list of channels:\n', list_channels)

if min_date is not None: min_date = pd.to_datetime(min_date).tz_localize('UTC').tz_convert('Europe/Madrid')
if max_date is not None: max_date = pd.to_datetime(max_date).tz_localize('UTC').tz_convert('Europe/Madrid')

In [ ]:
display(HTML('<h2>Data</h2>'))
display(HTML('<p><strong>Test name:</strong><br> {}</p>'.format(dispersion_test)))
display(HTML('<p><strong>Number of devices tested:</strong><br> {}</p>'.format(len(list_devices))))
display(HTML('<p><strong>Test dates:</strong><br> {} - {}</p>'.format(min_date, max_date)))
display(HTML('<p><strong>Test author(s):</strong><br> {}</p><hr>'.format('Óscar González - Victor Barberán')))

## Test

### Fraunhofer delivery
Devices with IDs ['10423', '10424', '10425', '10426', '10427', '10429', '10430', '10431']
### ICS delivery:
Devices with IDs 10474 to 10476
### iSCAPE Stations: 
Devices with IDs ['10408', '10409', '10412', '10413', '10414', '10420', '10421', '10422']

In [ ]:
list_isglobal = ['9974']
list_fraunhofer = ['10423', '10424', '10475', '10426', '10427', '10429', '10430', '10431']
list_ics = ['10474', '10425', '10476']
list_ecopenta = ['10557', '10558']
list_stations = ['10408', '10409', '10412', '10413', '10414', '10420', '10421', '10422']

In [ ]:
display(HTML('<h3>Test Explanation</h3>'))
display(HTML('<p>The methods used in this analysis are subject to the low amount of devices tested, and in general, a <i>common sense</i> approach is applied. This means that the plots indicate only trends and highlight some (but not all) potential faulty devices.</p>'))
display(HTML('<p>Below, the confidence intervals used are those of the normal distribution (meaningful for sample numbers >30) and of the t-student distribution.</p>'))
display(HTML('<p>Conclusions are derived by analysing the data further and testing the devices for longer periods if needed. Please, also note that statistical significance cannot be inferred from these amounts of devices, so some assumptions for normality have to be accepted. Comments and further discussion are always welcome.</p>'))
display(HTML('<p>Finally, the individual sensors components integrated in the Smart Citizen hardware have their own accuracies and dispersions, for which Smart Citizen cannot assume any liability other than trying to work with the most appropiate selection. The tests we perform are aimed to determine and assume any failures in the sensors and their integration within the Smart Citizen hardware. For more information, please check the <a href="https://docs.smartcitizen.me">official documentation</a> and the datasheets of each of the sensors.</p>'))

In [ ]:
list_devices = list_ics + list_ecopenta
list_devices

In [ ]:
from scipy import stats
from scipy.stats import t
import numpy as np
import traceback

# Calculate the dispersion for the sensors present in the dataset
dispersion_df = pd.DataFrame()
dispersion_history = list()
display(HTML('<h3>Warnings</h3>'))
warning_displayed = False
location = None

for device in list_devices:
    location_test = data.tests[dispersion_test].devices[device].location
    if location_test is None: data.std_out (f'Device {device} has no location')
    else:
        if location is None: location = location_test
        elif location_test != location: data.std_out (f'Device {device} has different location!')
        
for channel in list_channels:
    list_columns = list()
    if channel != 'BATT':
        for device in list_devices:
            if channel in data.tests[dispersion_test].devices[device].readings.columns and len(data.tests[dispersion_test].devices[device].readings.loc[:,channel]) >0 :
                # Important to resample and bfill for unmatching measures
                if smooth_channels:
                    channel_new = data.tests[dispersion_test].devices[device].readings[channel].resample('1Min').bfill().rolling(window=smooth_number).mean()
                    dispersion_df[channel + '-' + device] = channel_new[channel_new > 0]
                else:
                    dispersion_df[channel + '-' + device] = data.tests[dispersion_test].devices[device].readings[channel].resample('1Min').bfill()
                
                list_columns.append(channel + '-' + device)
            else:
                if device not in list_stations: continue
                display(HTML('<p>WARNING: Device {} does not contain {}</p>'.format(device, channel)))
                warning_displayed = True
    try:
        if dispersion_df.index.tzinfo is None: dispersion_df.index = dispersion_df.index.tz_localize('UTC').tz_convert(location)
    except:
        traceback.print_exc()
        pass
    
    # Trim dataset to min and max dates (normally these tests are carried out with _minutes_ of differences)
    if min_date is not None: dispersion_df = dispersion_df[dispersion_df.index > min_date]
    if max_date is not None: dispersion_df = dispersion_df[dispersion_df.index < max_date]

    # Calculate Metrics
    dispersion_df[channel + '_AVG'] = dispersion_df.loc[:,list_columns].mean(skipna=True, axis = 1)
    dispersion_df[channel + '_STD'] = dispersion_df.loc[:,list_columns].std(skipna=True, axis = 1)
 
    # Calculate Metrics
    dispersion_global = dispersion_df[channel + '_STD'].mean()
    # print (dispersion_df.index[0], dispersion_df.index[-1], channel, dispersion_global)
    dispersion_history.append([channel, dispersion_global])
if not warning_displayed:
    display(HTML('<p>All devices show similar amounts of data. No data loss concern</p>'.format(device, channel)))
    
# display(HTML('<h3>Sensor dispersion</h3>'))
# display(HTML('<p>Below, the sensor dispersion for each channel is listed (units of each sensor)</p>'))
# dispersion_history = tuple(dispersion_history)
# display(HTML('<ul style="list-style-type:disc;">'))
# for item in dispersion_history:
#     display(HTML('<li>{}: {}</li>'.format(item[0], item[1])))
# display(HTML('</ul>'))


In [ ]:
display(HTML('<hr>'))

### Internal notes:

+ NG PM Sensors: 10408, 10409, 10413, 10420, 10425, 10429:
    - 10425, 10429: replaced and tested for longer (from 11-11-2019 at 13:23 onwards)
    - 10425 replaced again 20/11 -> goes for ICS, 10475 is now part of Fraunhofer
    
+ iSCAPE:
    - 10408: pm sensor ??
    - 10413: pm sensor A
    - 10420: pm sensor A 
    - 10409: pm sensor B
     
    - Sensor resetting issue:
        1. 10422 1 PM Disconnected (11-11 19h)
        2. 10421 Gases Pro Board Disconnected (11-11 19h)
        3. Added battery to 9994 (12-11 13h)

## Conclusions

+ Devices show normal behaviour. Two warnings are displayed for the tVOC and eCO2 sensors, but are due to the sensor stabilisation period.
+ One PM sensor was replaced prior to 17/Nov due to sensor malfunction and shows proper behaviour. The warning displayed for the PM_25 metric is not considered abnormal sensor behaviour.

In [ ]:
display(HTML('<hr>'))

## Time Series Plot

In [ ]:
display(HTML('<h2>Time Series Plots</h2>'))
if min_date is None: min_date_test = dispersion_df.index[-1] 
else: min_date_test = min_date
if max_date is None: max_date_test = dispersion_df.index[-1] 
else: max_date_test = max_date
display(HTML('Min Date available: {}'.format(min_date_test)))
display(HTML('Max Date available: {}'.format(max_date_test)))
list_channels_kit =  ['PRESS', 'CCS811_ECO2', 'EXT_PM_10', 'NOISE_A', 'TEMP', 'CCS811_VOCS', 'HUM', 'EXT_PM_1', 'LIGHT', 'EXT_PM_25']

test_for_kit = True
list_channels_plots = list_channels_kit if test_for_kit else list_channels

# Ignore battery
if 'BATT' in list_channels_plots: list_channels_plots.remove('BATT')
dict_devices_tbr = dict()
for item in list_channels_plots: dict_devices_tbr[item] = list()

for channel in list_channels_plots:
    if channel not in list_channels_plots and test_for_kit: continue
    # Make subplot
    list_columns = list()
    fig, (ax1, ax2) = plot.subplots(nrows = 2, figsize= (15,10))
    cmap = plot.cm.Reds
    norm = matplotlib.colors.Normalize(vmin=0, vmax=limit_errors/2)
    index = list_channels_plots.index(channel)+1
    total_number = len(list_channels_plots)
    display(HTML('<h3>({}/{}) - {} </h3>'.format(index, total_number, channel)))
    
    dispersion_avg = 0
    limit_confidence_sigma = 0
    for item in dispersion_history:
        if channel == item[0]:
            dispersion_avg = item[1]
            
    if len(list_devices)>30:
        display(HTML('<p>Using Normal Distribution. Using limit for sigma confidence: {}'.format(limit_confidence_sigma)))
        limit_confidence = limit_confidence_sigma
        # Calculate upper and lower bounds
        if (use_instantatenous_dispersion):
            # For sensors with high variability in the measurements, it's better to use this (i.e. alphasense)
            upper_bound = dispersion_df[channel + '_AVG'] + limit_confidence * dispersion_df[channel + '_STD']
            lower_bound = dispersion_df[channel + '_AVG'] - abs(limit_confidence * dispersion_df[channel + '_STD'])
        else:
            upper_bound = dispersion_df[channel + '_AVG'] + limit_confidence * dispersion_avg
            lower_bound = dispersion_df[channel + '_AVG'] - abs(limit_confidence * dispersion_avg)
    else:
        limit_confidence = t.interval(t_confidence_level/100.0, len(list_devices), loc=dispersion_df[channel + '_AVG'], scale=dispersion_avg)
        display(HTML('<p>Using t-student Distribution</p>'))
        upper_bound = limit_confidence[1]
        lower_bound = limit_confidence[0]

    dispersion_df[channel + '_MAX'] = dispersion_df.loc[:,list_columns].max(skipna=True, axis = 1)
    dispersion_df[channel + '_MIN'] = dispersion_df.loc[:,list_columns].min(skipna=True, axis = 1)
        
    # print ('Plotting devices')
    for device in list_devices:
        name_column = channel + '-' + device 
        if name_column in dispersion_df.columns:
            # Count how many times we go above the upper bound or below the lower one
            count_problems_up = dispersion_df[name_column] > upper_bound
            count_problems_down =  dispersion_df[name_column] < lower_bound
            
            # Count them
            count_problems = [1 if (count_problems_up[i] or count_problems_down[i]) else 0 for i in range(len(count_problems_up))]
            # print (channel, device, np.sum(count_problems), len(count_problems))
            
            # Add the trace in either
            number_errors = np.sum(count_problems)
            max_number_errors = len(count_problems)
            
            if number_errors/max_number_errors > limit_errors/100:
                print (f'WARNING: Device {device} out of {limit_errors}% limit - {np.round(number_errors/max_number_errors*100, 1)}% out')
                if device not in dict_devices_tbr[channel]: dict_devices_tbr[channel].append(device)
                alpha = 1
                ax1.plot(dispersion_df.index, 
                         dispersion_df[name_column], 
                         color = 'r',
                         label = device, alpha = alpha)
            else:
                alpha = 1
                color = 'g'
                ax2.plot(dispersion_df.index, 
                         dispersion_df[name_column], 
                         color = color, 
                         label = device, alpha = alpha)
        
    # Add upper and low bound bound to subplot 1
    ax1.plot(dispersion_df.index, dispersion_df[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax1.plot(dispersion_df.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax1.plot(dispersion_df.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    lgd1 = ax1.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax1.grid(True)
    ax1.set_ylabel(channel + ' TBR')
    ax1.set_xlabel('Time')
    
    # Add upper and low bound bound to subplot 2
    ax2.plot(dispersion_df.index, dispersion_df[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax2.plot(dispersion_df.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax2.plot(dispersion_df.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    lgd2 = ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax2.grid(True)
    ax2.set_ylabel(channel + ' OK')
    ax2.set_xlabel('Time')
    
    # Check file type to make the export
    if type_file is not None: print ('Saving figure')
    if type_file == 'fig':
        pickle.dump(fig, open(save_path + '/' + dispersion_test + '_' + channel + '.fig.pickle', 'wb'))
    elif type_file == 'png':
        fig.savefig(save_path + '/' + dispersion_test + '_' + channel + '.png', dpi=300, trasnparent = True, bbox_extra_artists=(lgd1, lgd2), bbox_inches='tight' )

    # Show plots     
    plot.show()
    display(HTML('<br>'))

### Summary

In [ ]:
summary_df = pd.DataFrame(index = list_channels_plots)

for item in dispersion_history:
    summary_df.loc[item[0], 'Dispersion'] = item[1]
    if item[0] != 'BATT':
        summary_df.loc[item[0], 'Total Number of devices'] = len(list_devices)
        summary_df.loc[item[0], 'TBR Number of devices'] = len(dict_devices_tbr[item[0]])
        summary_df.loc[item[0], 'OK Number of devices'] = len(list_devices) - len(dict_devices_tbr[item[0]])
    
display (summary_df)

# Further testing

In [ ]:
plot_description = {"plot_type": "timeseries",
                    "plotting_library": "plotly",
                    "data": {"test": "2019-11_INT_8_FRAUNHOFER_8_STATION_ISCAPE",
									"traces": {"1": {"device": "all",
													"channel" : "EXT_PM_25",
													"subplot": 1}}},
                    "options": {"clean_na": None,
                                "show_plot": True,
                                "separate_device_plots": False,
                                "export_path": "/Users/macoscar/Desktop", 
                                "file_name": "PM_SENSOR_ISSUE.png",
                                "target_raster": '10Min',
                                "max_date": '2019-11-23',
                                "min_date": '2019-11-17'},
                    "formatting": {"xlabel": "Date (-)",
                                   "ylabel": {1: "EXT_PM_25"},
                                   "yrange": {1: [-10, 180]},
                                   "title": "EXT_PM_25 - PM Sensor issue",
                                   "sharex":True,
                                   "grid": True,
                                   "height": 10,
                                   "width": 15}
                    }

from src.visualization.visualization import plot_wrapper

plot_object = plot_wrapper(plot_description, False)
plot_object.plot(records)
plot_object.export_plot()

In [ ]:
display(HTML('<h2>Time Series Plots</h2>'))
display(HTML('Min Date available: {}'.format(min_date)))
display(HTML('Max Date available: {}'.format(max_date)))
        
# Ignore battery
if 'BATT' in list_channels: list_channels.remove('BATT')
dict_devices_tbr = dict()
for item in list_channels: dict_devices_tbr[item] = list()

for channel in list_channels:
    # Make subplot
    list_columns = list()
    fig, (ax1, ax2) = plot.subplots(nrows = 2, figsize= (15,10))
    cmap = plot.cm.Reds
    norm = matplotlib.colors.Normalize(vmin=0, vmax=limit_errors/2)
    display(HTML('<h3>({}/{}) - {} </h3>'.format(list_channels.index(channel)+1, len(list_channels), channel)))
    
    dispersion_avg = 0
    limit_confidence_sigma = 0
    for item in dispersion_history:
        if channel == item[0]:
            dispersion_avg = item[1]
            
    if len(list_devices)>30:
        display(HTML('<p>Using Normal Distribution. Using limit for sigma confidence: {}'.format(limit_confidence_sigma)))
        limit_confidence = limit_confidence_sigma
        # Calculate upper and lower bounds
        if (use_instantatenous_dispersion):
            # For sensors with high variability in the measurements, it's better to use this (i.e. alphasense)
            upper_bound = dispersion_df[channel + '_AVG'] + limit_confidence * dispersion_df[channel + '_STD']
            lower_bound = dispersion_df[channel + '_AVG'] - abs(limit_confidence * dispersion_df[channel + '_STD'])
        else:
            upper_bound = dispersion_df[channel + '_AVG'] + limit_confidence * dispersion_avg
            lower_bound = dispersion_df[channel + '_AVG'] - abs(limit_confidence * dispersion_avg)
    else:
        limit_confidence = t.interval(t_confidence_level/100.0, len(list_devices), loc=dispersion_df[channel + '_AVG'], scale=dispersion_avg)
        display(HTML('<p>Using t-student Distribution</p>'))
        upper_bound = limit_confidence[1]
        lower_bound = limit_confidence[0]

    dispersion_df[channel + '_MAX'] = dispersion_df.loc[:,list_columns].max(skipna=True, axis = 1)
    dispersion_df[channel + '_MIN'] = dispersion_df.loc[:,list_columns].min(skipna=True, axis = 1)
        
    # print ('Plotting devices')
    for device in list_devices:
        name_column = channel + '-' + device 
        if name_column in dispersion_df.columns:
            # Count how many times we go above the upper bound or below the lower one
            count_problems_up = dispersion_df[name_column] > upper_bound
            count_problems_down =  dispersion_df[name_column] < lower_bound
            
            # Count them
            count_problems = [1 if (count_problems_up[i] or count_problems_down[i]) else 0 for i in range(len(count_problems_up))]
            # print (channel, device, np.sum(count_problems), len(count_problems))
            
            # Add the trace in either
            number_errors = np.sum(count_problems)
            max_number_errors = len(count_problems)
            
            if number_errors/max_number_errors > limit_errors/100:
                print (f'WARNING: Device {device} out of {limit_errors}% limit - {np.round(number_errors/max_number_errors*100, 1)}% out')
                if device not in dict_devices_tbr[channel]: dict_devices_tbr[channel].append(device)
                alpha = 0.6
                ax1.plot(dispersion_df.index, 
                         dispersion_df[name_column], 
                         color = cmap(norm(number_errors/max_number_errors*limit_errors)),
                         label = device, alpha = alpha)
            else:
                alpha = 0.6
                color = 'g'
                ax2.plot(dispersion_df.index, 
                         dispersion_df[name_column], 
                         color = color, 
                         label = device, alpha = alpha)
        
    # Add upper and low bound bound to subplot 1
    ax1.plot(dispersion_df.index, dispersion_df[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax1.plot(dispersion_df.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax1.plot(dispersion_df.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    lgd1 = ax1.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax1.grid(True)
    ax1.set_ylabel(channel + ' TBR')
    ax1.set_xlabel('Time')
    
    # Add upper and low bound bound to subplot 2
    ax2.plot(dispersion_df.index, dispersion_df[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax2.plot(dispersion_df.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax2.plot(dispersion_df.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    lgd2 = ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax2.grid(True)
    ax2.set_ylabel(channel + ' OK')
    ax2.set_xlabel('Time')
    
    # Check file type to make the export
    if type_file is not None: print ('Saving figure')
    if type_file == 'fig':
        pickle.dump(fig, open(save_path + '/' + dispersion_test + '_' + channel + '.fig.pickle', 'wb'))
    elif type_file == 'png':
        fig.savefig(save_path + '/' + dispersion_test + '_' + channel + '.png', dpi=300, trasnparent = True, bbox_extra_artists=(lgd1, lgd2), bbox_inches='tight' )

    # Show plots     
    plot.show()
    display(HTML('<br>'))